In [1]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.geometry import MultiLineString

In [2]:
stations_dir = "../Data/stations.csv"
lines_def_dir = "../Data/lines_definition.csv"
route_dir = "../Data/routes.csv"

In [3]:
tube_df = pd.read_csv(stations_dir, index_col='id')
lines_df = pd.read_csv(lines_def_dir)
routes_df = pd.read_csv(route_dir)



In [4]:
tube_df['geometry'] = None
for index, row in tube_df.iterrows():
    tube_df.loc[index, ['geometry']] = Point(row['latitude'], row['longitude'])
tube_df.pop('latitude')
tube_df.pop('longitude')
tube_df = gpd.GeoDataFrame(tube_df)

In [5]:
lines_gdf = gpd.GeoDataFrame({"line": [], "geometry":[]}, geometry="geometry")
lines_gdf.line = list(lines_df.line.unique())
lines_gdf = lines_gdf.set_index("line")

In [6]:
lines_df['geometry'] = None

for index, row in lines_df.iterrows():
 
    lines_df.loc[index, ['geometry']] = LineString([tube_df.loc[row.station1, ['geometry']].values[0], tube_df.loc[row.station2, ['geometry']].values[0]])
    
routes_df['geometry'] = None


In [7]:
for index, row in routes_df.iterrows():
    lines_tube = lines_df.loc[lines_df['line'] == row["line"], ['geometry']].values.tolist()
    lines_aux = []
    for n in lines_tube:
        lines_aux.append(n.pop())
    
    routes_df.loc[index, ['geometry']] = MultiLineString(lines_aux)

In [12]:
def zone_color(zone):
    zones = ['#006400' ,'#008000' ,'#00FF00' , #dark_green,  green,         light green
             '#FFFF00' ,'#FFA500' ,'#FF4500' , #yellow,      orange-yellow, orange
             '#FF3300' ,'#FF0000' ,'#8B0000']  #dark orange, light red,     dark red
    return zones[zone-1]



m_1 = folium.Map(location = [51.5074, -0.1278])

for index, row in routes_df.iterrows():
    colour = row.colour
    for line in row.geometry.geoms:
        folium.PolyLine(locations = list(line.coords), color = "#" + colour).add_to(m_1)
    
for index, row in tube_df.iterrows():
    folium.Circle(location = row.geometry.coords[0],radius=5, color = zone_color(int(tube_df.loc[index, ['zone']].values.tolist()[0]))).add_to(m_1)

In [ ]:
#Uncomment to show teh map on your browser
#m_1.show_in_browser()